In [1]:
from fastembed import TextEmbedding

## Q1. Embedding the query

In [2]:
query = 'I just discovered the course. Can I join now?'

embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")
query_embedding = embedder.embed([query])

query_vector = next(query_embedding)
print(query_vector.min())

-0.11726373885183883


Result Q1: -0.11

In [3]:
import numpy as np
np.linalg.norm(query_vector)

np.float64(1.0)

In [4]:
query_vector.dot(query_vector)

np.float64(1.0000000000000002)

## Q2. Cosine similarity with another vector

In [7]:
doc = 'Can I still join the course after the start date?'
doc_embedding = embedder.embed([doc])

doc_vector = next(doc_embedding)

In [9]:
dot_product = query_vector.dot(doc_vector)
dot_product

np.float64(0.9008528895674548)

## Question 3. Ranking by cosine

In [10]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [26]:
text_embeddings = list(embedder.embed([doc['text'] for doc in documents]))
V = np.stack(text_embeddings)

In [31]:
max_cosine = -1.0
max_index = -1

for i, v in enumerate(V):
    cosine = query_vector.dot(v)
    print(i, cosine)
    if cosine > max_cosine:
        max_cosine = cosine
        max_index = i

print(f'\nMax cosine similarity: {max_cosine} at index {max_index}\n')
print(query)
print(documents[max_index]["question"])
print(documents[max_index]["text"])

0 0.7629684696540238
1 0.8182378150042889
2 0.8085397398734189
3 0.7133079015686243
4 0.7304499234333598

Max cosine similarity: 0.8182378150042889 at index 1

I just discovered the course. Can I join now?
Course - Can I follow the course after it finishes?
Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.


Result: 1

## Q4. Ranking by cosine, version two

In [32]:
text_embeddings = list(embedder.embed([doc['question'] + ' ' + doc['text'] for doc in documents]))
V_question = np.stack(text_embeddings)

max_cosine = -1.0
max_index = -1

for i, v in enumerate(V_question):
    cosine = query_vector.dot(v)
    print(i, cosine)
    if cosine > max_cosine:
        max_cosine = cosine
        max_index = i

print(f'\nMax cosine similarity: {max_cosine} at index {max_index}\n')
print(query)
print(documents[max_index]["question"])
print(documents[max_index]["text"])

0 0.8514543236908068
1 0.8436594159113068
2 0.8408287048502558
3 0.7755157969663908
4 0.8086007795043938

Max cosine similarity: 0.8514543236908068 at index 0

I just discovered the course. Can I join now?
Course - Can I still join the course after the start date?
Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Result: 0

## Q5. Selecting the embedding model

In [34]:
dir(TextEmbedding)

['EMBEDDINGS_REGISTRY',
 'METADATA_FILE',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_model_description',
 '_is_protocol',
 '_list_supported_models',
 'add_custom_model',
 'decompress_to_cache',
 'download_file_from_gcs',
 'download_files_from_huggingface',
 'download_model',
 'embed',
 'list_supported_models',
 'passage_embed',
 'query_embed',
 'retrieve_model_gcs']

In [37]:
print(min([model['dim'] for model in TextEmbedding.list_supported_models()]))

384


Result: 384

## Q6. Indexing with qdrant (2 points)

In [38]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [39]:
embedder_bge_small = TextEmbedding(model_name="BAAI/bge-small-en")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

In [40]:
text_embeddings = list(embedder.embed([doc['question'] + ' ' + doc['text'] for doc in documents]))
V_bge_small = np.stack(text_embeddings)

In [43]:
max_cosine = -1.0
max_index = -1

for i, v in enumerate(V_bge_small):
    cosine = query_vector.dot(v)
    if cosine > max_cosine:
        max_cosine = cosine
        max_index = i

print(f'\nMax cosine similarity: {max_cosine} at index {max_index}\n')
print(query)
print(documents[max_index]["question"])
print(documents[max_index]["text"])


Max cosine similarity: 0.8620737682350726 at index 14

I just discovered the course. Can I join now?
The course has already started. Can I still join it?
Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.
